# CPSC 330 midterm

The University of British Columbia

Instructor: Mike Gelbart

February 13, 2020

Please write your CS ID (a.g. a1b2c) **clearly and legibly** at the left-hand side of the box below:

In [2]:
cs_id = "p6n4n"

In this exam we'll be analyzing the [Rain in Australia](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) dataset from Kaggle. This exam consists of 14 **equally weighted** questions interspersed between parts of the analysis. The exam was printed directly from Jupyter; that is, all the output you see was generated by the code you see, with no "tampering" after-the-fact.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 16
import graphviz
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score 
# plot_confusion_matrix is deprecated
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [4]:
import sys
import sklearn
print(sys.executable)
print(sklearn.__version__)

C:\Users\joshu\miniconda3\envs\cpsc330\python.exe
1.3.2


In [5]:
# pd.set_option('precision', 2)
# pd.set_option('precision', 2)

The last line (above) tells pandas to print numbers to 2 decimal places (the default is 6 places). This should improve readability for you.

## Preliminaries

Load the data:

In [6]:
df = pd.read_csv('./data/weatherAUS.csv', parse_dates=['Date'])

Split the data by date (you can assume this is a reasonable thing to do):

In [7]:
df_train = df.query('Date <= 20121231')
df_valid = df.query('20130101 <= Date <= 20151231')
df_test  = df.query('Date >= 20160101')

First, I call `df.describe()` to get a sense of some of the features:

In [8]:
df.describe()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,145460,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,2013-04-04 21:08:51.907053568,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
min,2007-11-01 00:00:00,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,2011-01-11 00:00:00,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,2013-06-02 00:00:00,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,2015-06-14 00:00:00,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,2017-06-25 00:00:00,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000
std,NaN,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665


It's interesting to note that the max temperature was 48 degrees; that's very hot!

## Q1

What did I do wrong in the above line, and what should I have done instead? **Maximum 2 sentences.**

<br><br><br><br><br><br><br><br>

We violated the golden rule by looking at the test set ahead of time. 


---------------

Next I create X and y. Note: the dataset documentation says to remove `RISK_MM`. I will also drop the `Date`.

In [9]:
target_column = 'RainTomorrow'
drop_columns = ['Date']
X_train = df_train.drop(columns=[target_column] + drop_columns)
X_valid = df_valid.drop(columns=[target_column] + drop_columns)
X_test  = df_test.drop( columns=[target_column] + drop_columns)

In [10]:
y_train = df_train[target_column]
y_valid = df_valid[target_column]
y_test = df_test[target_column]

Start looking at the data:

In [11]:
X_train.shape

(66718, 21)

In [12]:
X_train

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140690,Darwin,25.0,35.4,0.0,7.4,11.7,N,39.0,NNE,N,...,20.0,57.0,42.0,1012.3,1008.1,2.0,2.0,31.4,34.3,No
140691,Darwin,26.5,35.9,0.0,8.0,10.3,N,35.0,E,NNW,...,20.0,65.0,40.0,1010.3,1007.3,6.0,3.0,31.4,35.1,No
140692,Darwin,27.4,35.0,0.0,7.8,6.5,WNW,31.0,NE,NW,...,17.0,69.0,48.0,1011.0,1007.8,6.0,7.0,29.9,33.7,No
140693,Darwin,24.8,33.5,3.4,7.4,4.7,E,43.0,ENE,W,...,15.0,84.0,75.0,1010.8,1007.6,7.0,7.0,27.3,29.0,Yes


In [13]:
numeric_features = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']
categorical_features = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

Make sure all columns are accounted for:

In [14]:
all_features = numeric_features + categorical_features

In [15]:
assert set(X_train.columns) == set(all_features)

Next, I perform similar transformations to what you've seen in the course, using `SimpleImputer`, `StandardScaler`, and `OneHotEncoder`. 

In [16]:
imputers = [('num', SimpleImputer(strategy='median'), numeric_features),
    ('cat', SimpleImputer(strategy='most_frequent'), categorical_features)]

In [17]:
impute_transformer = ColumnTransformer(transformers=imputers)

In [18]:
impute_transformer.fit(X_train);

In [19]:
X_train_imp = pd.DataFrame(impute_transformer.transform(X_train), index=X_train.index, columns=all_features)
X_valid_imp = pd.DataFrame(impute_transformer.transform(X_valid), index=X_valid.index, columns=all_features)
X_test_imp  = pd.DataFrame(impute_transformer.transform(X_test),  index=X_test.index,  columns=all_features)

Below, we use `drop='first'` for all the columns except 'Location':

In [20]:
transformers=[
    ('scale', StandardScaler(), numeric_features),
    ('ohe1', OneHotEncoder(sparse=False, handle_unknown='ignore'), ['Location']),
    ('ohe2', OneHotEncoder(sparse=False, drop='first'), categorical_features[1:])]

In [21]:
preprocessor = ColumnTransformer(transformers=transformers)

In [22]:
preprocessor.fit(X_train_imp);

C:\Users\joshu\miniconda3\envs\cpsc330\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\joshu\miniconda3\envs\cpsc330\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [23]:
new_columns = numeric_features + list(preprocessor.named_transformers_['ohe1'].get_feature_names_out(['Location'])) + list(preprocessor.named_transformers_['ohe2'].get_feature_names_out(categorical_features[1:]))

In [24]:
X_train_transformed = pd.DataFrame(preprocessor.transform(X_train_imp), index=X_train_imp.index, columns=new_columns)
X_valid_transformed = pd.DataFrame(preprocessor.transform(X_valid_imp), index=X_valid_imp.index, columns=new_columns)
X_test_transformed  = pd.DataFrame(preprocessor.transform(X_test_imp),  index=X_test_imp.index,  columns=new_columns)

## Q2

The `RainToday` feature is binary: it only takes on the values "Yes" or "No". I used `OneHotEncoder` to one-hot encode this feature with `drop='first'`. What differences we would observe, if any, had we instead used `OrdinalEncoder` for `RainToday`? Briefly justify your answer. **Maximum 2 sentences.**

<br><br><br><br><br><br><br><br>


There would be no differences observed. 

---------------

Next, I'll use a `DummyClassifier` like the one you implemented in hw4:

In [28]:
y_train

0          No
1          No
2          No
3          No
4          No
         ... 
140690     No
140691     No
140692    Yes
140693     No
140694     No
Name: RainTomorrow, Length: 66718, dtype: object

In [27]:
dc = DummyClassifier(strategy='prior')
dc.fit(X_train_transformed, y_train);

TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
dc.score(X_train_transformed, y_train)

In [ ]:
dc.score(X_valid_transformed, y_valid)

## Q3

Given the above, quantify the level of class imbalance in this dataset. Briefly justify your answer. **Maximum 2 sentences.**

<br><br><br><br><br><br><br><br>

---------------

Next, we fit a `DecisionTreeClassifier` where the maximum depth of the tree is 2. We set `max_features=5` only because it makes the exam question more interesting, not because it's a good idea.

In [29]:
dt = DecisionTreeClassifier(max_depth=2, max_features=5, random_state=321)
dt.fit(X_train_transformed, y_train);

ValueError: Input contains NaN

We then visualize the tree:

In [ ]:
graphviz.Source(export_graphviz(dt, out_file=None, feature_names=new_columns,
                                class_names=dt.classes_, impurity=False))

Note: for some reason, the arrows are not labeled on the second level, but you can assume that the left side branch is always True and the right branch is always False. So, the 4 arrows in the second level should be  labelled True, False, True, False if reading from left to right.

Here is the first element of the validation set. First I print out (a subset of) the **transformed** numerical variables:

In [ ]:
X_valid_transformed[numeric_features].iloc[:1,9:]

And then I print out the **un-transformed** categorical features:

In [ ]:
X_valid[categorical_features].iloc[:1]

## Q4

Describe the path of this example down the tree. Which of the four bottom nodes does it end up at: the 1st, 2nd, 3rd or 4th? **Maximum 3 sentences.**

<br><br><br><br><br><br><br><br><br><br><br>

---------------

The following code grabs 4 numeric features - but I'm not telling you which ones.

In [ ]:
np.random.seed(1) # set random state
some_numeric_features = np.random.permutation(numeric_features)[10:14]

In hw2, for the spotify dataset, you looked at histograms of various feature values split up by target class. The following code creates 4 such histograms, one for each of the 4 randomly chosen numeric features, using the training set.

In [ ]:
X_train_neg = X_train_transformed[y_train == "No"]
X_train_pos = X_train_transformed[y_train == "Yes"]

plt.figure(figsize=(10,10))
for i, feature in enumerate(some_numeric_features):

    plt.subplot(2,2,i+1)
    plt.hist(X_train_neg[feature], alpha=0.6, bins=30, label="No", hatch="-")
    plt.hist(X_train_pos[feature], alpha=0.3, bins=30, label="Yes", hatch="*")
    plt.legend(loc='upper right')
    plt.xlabel('some feature')
    plt.ylabel("count")
    plt.title(f"Histogram {i+1}")
plt.tight_layout();

## Q5

I haven't told you which feature is being used to generate each histogram. Based on the decision tree visualization above, which of the 4 histograms (Histogram 1, Histogram 2, Histogram 3, or Histogram 4) do you think corresponds to the feature `Humidity3pm`? Briefly justify your answer. **Maximum 3 sentences.**

Hint: `Humidity3pm <= 0.859` was chosen by `DecisionTreeClassifier` to be the very first split (i.e., it's at the top of the tree). This indicates that it's a useful split for separating the classes.

<br><br><br><br><br><br><br><br><br><br><br>

---------------

Next, I'll fit a logistic regression:

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train_transformed, y_train);

In [ ]:
lr.score(X_train_transformed, y_train)

In [ ]:
lr.score(X_valid_transformed, y_valid)

## Q6

Given the above, would it make more sense to try increasing or decreasing the `C` hyperparameter? As a reminder, increasing `C` increases model complexity. Briefly justify your answer. **Maximum 2 sentences.**



<br><br><br><br><br><br><br><br>

---------------

The following are some of the coefficients learned by `LogisticRegression` (only some of them are shown):

In [ ]:
pd.DataFrame(data=lr.coef_[0], index=X_train_transformed.columns, columns=["Coefficient"])

## Q7

You will notice that `Evaporation` has a positive coefficient. True or False: increasing the `Evaporation` value (and leaving all other features fixed) will increased the predicted probability that it will rain tomorrow (i.e., the positive class). Briefly justify your answer. **Maximum 2 sentences.**

<br><br><br><br><br><br><br><br>

---------------

## Q8

You will notice that `WindDir3pm_W`, which is a binary variable created by the `OneHotEncoder`, has a positive coefficient. True or False: an observation with `WindDir3pm` equal to 'W' will have a higher  predicted probability that it will rain tomorrow (i.e., the positive class) than an observation with `WindDir3pm` equal to any other value (leaving all other original features fixed). Briefly justify your answer. **Maximum 2 sentences.**

<br><br><br><br><br><br><br><br>

---------------

Next I will train a random forest on this dataset:

In [ ]:
rf = RandomForestClassifier(random_state=123)
rf.fit(X_train_transformed, y_train);

Here's the confusion matrix based on the **training** set:

In [ ]:
disp = plot_confusion_matrix(rf, X_train_transformed, y_train,
                             display_labels=rf.classes_,
                             cmap=plt.cm.Blues, 
                             values_format = 'd');

## Q9

Suppose we re-ran the code but with `class_weight="balanced"`. In that case, would you expect the number in the **lower-left** box to increase or decrease? Briefly justify your answer. **Maximum 3 sentences.**

Hint: as a reminder, `class_weight="balanced"` increases the "weight" (or importance) of examples in the minority class.

<br><br><br><br><br><br><br><br><br><br><br>

---------------

Below is the confusion matrix for the **validation** set.

In [ ]:
disp = plot_confusion_matrix(rf, X_valid_transformed, y_valid,
                             display_labels=rf.classes_,
                             cmap=plt.cm.Blues, 
                             values_format = 'd');

## Q10

Would you say this classifier is overfit? Briefly justify your answer. **Maximum 2 sentences.**

<br><br><br><br><br><br><br><br>

---------------

In scikit-learn the `RandomForestClassifier` also has a `predict_proba` method, so we can create an ROC curve. Below I do so on the **validation** set:

In [ ]:
fpr, tpr, thresholds = roc_curve(y_valid, rf.predict_proba(X_valid_transformed)[:,1], pos_label="Yes")

plt.plot(fpr, tpr);
plt.plot((0,1),(0,1),'--k');
plt.xlabel('false positive rate');
plt.ylabel('true positive rate\n(a.k.a. recall)');

## Q11

On the plot above, draw a circle to mark the point where the probability cutoff threshold is equal to 0.5. Briefly justify your answer. **Maximum 3 sentences.**

<br><br><br><br><br><br><br><br><br><br><br>

---------------

The following code grabs three classifiers learned above:
  - The `DummyClassifier` called `dc`.
  - The `LogisticRegression` called `lr`.
  - The `RandomForestClassifier` called `rf`.

In [ ]:
some_models = [dc, lr, rf] 

Next, I shuffle the list of models so you don't know what order they are in.

In [ ]:
np.random.seed(999)
some_models = np.random.permutation(some_models)

To remind you, here is the training accuracy in each case:

In [ ]:
dc.score(X_train_transformed, y_train)

In [ ]:
rf.score(X_train_transformed, y_train)

In Lecture 8 we looked at some histograms of the predicted probabilities, split by the true class. The following code creates 3 such histograms for 3 different models, using the training set.

In [ ]:
plt.figure(figsize=(10,10))
for i, model in enumerate(some_models):
    plt.subplot(2,2,i+1)
    plt.hist(model.predict_proba(X_train_neg)[:,1], alpha=0.6, bins=50, label="No", hatch="-")
    plt.hist(model.predict_proba(X_train_pos)[:,1], alpha=0.3, bins=50, label="Yes", hatch="*")
    plt.legend(loc='upper right')
    plt.xlabel('predicted probability')
    plt.ylabel("count")
    plt.title(f"Histogram {i+1}")
plt.tight_layout();

## Q12

I haven't told you which model is being used to generate each histogram. Which of the 3 histograms (Histogram 1, Histogram 2, or Histogram 3) do you think corresponds to the **logistic regression** (`lr`) model? Briefly justify your answer. **Maximum 3 sentences.**

<br><br><br><br><br><br><br><br><br><br><br>

---------------

Next, I want to optimize the hyperparameters of my random forest. 

In [ ]:
param_grid = {
              "n_estimators"     : [10,100],
              "max_depth"        : [3, 5, 10, 15, None],
              "max_features"     : [3, None]
             }
param_grid

In [ ]:
grid_search = GridSearchCV(RandomForestClassifier(), 
                           param_grid, cv=5, refit=False,
                           verbose=5, n_jobs=-1)

## Q13

For the grid search above, how many models are trained in total? Briefly justify your answer. **Maximum 2 sentences.**

Note: Don't forget to account for both the different hyperparameter values and the cross-validation folds. Also, I set `refit=False` so you don't have to account for one extra model being trained at the end.

<br><br><br><br><br><br><br><br>

---------------

At the next point of my analysis, I experience temporary insanity. I do several crazy things:

1. I set all relevant hyperparameters to 3, because 3 is a really cool number.
2. I decide to just optimize `random_state` using `GridSearchCV`, trying all integers from 0 to 999 for the `random_state`.
3. I decide to just use the first 100 training examples for my cross-validation, because I'm too impatient to wait any longer.

In [ ]:
grid_search = GridSearchCV(RandomForestClassifier(n_estimators=3, max_depth=3, max_features=3), 
                           {"random_state" : list(range(1000))}, 
                           cv=3, verbose=0, n_jobs=-1)

In [ ]:
grid_search.fit(X_train_transformed[:100], y_train[:100]);

Upon doing so, I get a fantastic cross-validation score:

In [ ]:
grid_search.best_score_

I will now test out this model on the test set:

In [ ]:
grid_search.score(X_test_transformed, y_test)

## Q14

Explain why the test accuracy is so much worse than the cross-validation score. **Maximum 3 sentences.**

<br><br><br><br><br><br><br><br><br><br><br>

---------------